In [28]:
import pandas as pd
from sklearn.model_selection import train_test_split

training_data = pd.read_json('C:/Users/koari/OneDrive/Desktop/IE4483 Proj/train.json')
training_data = pd.DataFrame(training_data)
print(training_data)

train_texts, test_texts, train_labels, test_labels = train_test_split(training_data['reviews'], training_data['sentiments'], test_size=0.25, random_state=42)

                                                reviews  sentiments
0     I bought this belt for my daughter in-law for ...           1
1     The size was perfect and so was the color.  It...           1
2     Fits and feels good, esp. for doing a swim rac...           1
3     These socks are absolutely the best. I take pi...           1
4     Thank you so much for the speedy delivery they...           1
...                                                 ...         ...
7396  I bought these shirts (black, medium) to wear ...           0
7397  At first, I thought this scarf might not be th...           1
7398  I am very picky when it comes to bras.  I want...           1
7399  This jacket is wind and water resistant, but n...           0
7400  These are extremely confortable. The material ...           1

[7401 rows x 2 columns]


In [29]:
from transformers import BertTokenizer, BertForSequenceClassification

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [30]:
import torch

# Tokenize the reviews and convert them into BERT's input format
train_encodings = tokenizer(list(train_texts), truncation=True, padding=True, max_length=128)
test_encodings = tokenizer(list(test_texts), truncation=True, padding=True, max_length=128)

# Convert the labels into torch tensors
train_labels = torch.tensor(train_labels.tolist())
test_labels = torch.tensor(test_labels.tolist())

# Create a dataset class to load the data into PyTorch
class SentimentDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = self.labels[idx]
        return item

train_dataset = SentimentDataset(train_encodings, train_labels)
test_dataset = SentimentDataset(test_encodings, test_labels)


In [31]:
from transformers import Trainer, TrainingArguments

# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',          # output directory for model checkpoints
    num_train_epochs=3,              # number of training epochs
    per_device_train_batch_size=16,  # batch size for training
    per_device_eval_batch_size=64,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,
)

# Create the Trainer object
trainer = Trainer(
    model=model,                         # the pre-trained BERT model
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=test_dataset            # evaluation dataset
)

# Fine-tune the model
trainer.train()

  0%|          | 0/1041 [00:00<?, ?it/s]

{'loss': 0.7455, 'grad_norm': 13.063673973083496, 'learning_rate': 1.0000000000000002e-06, 'epoch': 0.03}
{'loss': 0.7149, 'grad_norm': 11.670839309692383, 'learning_rate': 2.0000000000000003e-06, 'epoch': 0.06}
{'loss': 0.6285, 'grad_norm': 7.790703773498535, 'learning_rate': 3e-06, 'epoch': 0.09}
{'loss': 0.5439, 'grad_norm': 2.470651626586914, 'learning_rate': 4.000000000000001e-06, 'epoch': 0.12}
{'loss': 0.4742, 'grad_norm': 3.8192412853240967, 'learning_rate': 5e-06, 'epoch': 0.14}
{'loss': 0.4246, 'grad_norm': 3.4432435035705566, 'learning_rate': 6e-06, 'epoch': 0.17}
{'loss': 0.3958, 'grad_norm': 5.625775337219238, 'learning_rate': 7.000000000000001e-06, 'epoch': 0.2}
{'loss': 0.4173, 'grad_norm': 4.716664791107178, 'learning_rate': 8.000000000000001e-06, 'epoch': 0.23}
{'loss': 0.4496, 'grad_norm': 3.0991733074188232, 'learning_rate': 9e-06, 'epoch': 0.26}
{'loss': 0.3704, 'grad_norm': 1.7843981981277466, 'learning_rate': 1e-05, 'epoch': 0.29}
{'loss': 0.3509, 'grad_norm': 2.8

TrainOutput(global_step=1041, training_loss=0.1707888268637511, metrics={'train_runtime': 4683.6253, 'train_samples_per_second': 3.555, 'train_steps_per_second': 0.222, 'total_flos': 1095199767936000.0, 'train_loss': 0.1707888268637511, 'epoch': 3.0})

In [32]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

# Predict on the test dataset
predictions = trainer.predict(test_dataset)

# Get the predicted labels
preds = torch.argmax(torch.from_numpy(predictions.predictions), axis=-1)

# Calculate accuracy, precision, recall, and F1-score
accuracy = accuracy_score(test_labels, preds)
precision, recall, f1, _ = precision_recall_fscore_support(test_labels, preds, average='binary')

print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1-score: {f1}")

  0%|          | 0/29 [00:00<?, ?it/s]

Accuracy: 0.9502971366828741
Precision: 0.9628252788104089
Recall: 0.9798234552332913
F1-score: 0.97125


In [33]:
# Save model and tokenizer
model.save_pretrained('./saved_model')
tokenizer.save_pretrained('./saved_model')

# Load model and tokenizer
tokenizer = BertTokenizer.from_pretrained('./saved_model')
model = BertForSequenceClassification.from_pretrained('./saved_model')

In [34]:
# Import test.json for prediction 
prediction_data = pd.read_json('C:/Users/koari/OneDrive/Desktop/IE4483 Proj/test.json')
prediction_data = pd.DataFrame(prediction_data)
print(prediction_data)

                                                reviews
0     I bought 2 sleepers.  sleeper had holes in the...
1     I dare say these are just about the sexiest th...
2     everything about the transaction (price, deliv...
3     Not bad for just a shirt.  Very durable, and m...
4     These are truly wrinkle free and longer than t...
...                                                 ...
1846  These are stylish and nice and perfect to wear...
1847  We bought these long-sleeved colored shirts fo...
1848  These are really cute and very sexy.  They wil...
1849  These shoers were for my daughter.  She loves ...
1850  Most umbrellas handle a light rain well, but l...

[1851 rows x 1 columns]


In [35]:
def predict_sentiment(review):
    # Tokenize the review
    inputs = tokenizer(review, return_tensors="pt", truncation=True, padding=True, max_length=128)
    
    # Perform the prediction
    outputs = model(**inputs)
    probs = torch.nn.functional.softmax(outputs.logits, dim=-1)
    # Return Sentiment Value (0 for negative, 1 for postive)
    print(torch.argmax(probs).item())
    return torch.argmax(probs).item()

# Stores the sentiments in a list
prediction_list = []
for row in range(len(prediction_data)):
    prediction_list.append(predict_sentiment(prediction_data.iloc[row][0]))

# Append the sentiment list to the a new dataframe
finalized_prediction = prediction_data.assign(sentiment=prediction_list)

0


C:\Users\koari\AppData\Local\Temp\ipykernel_11132\3902533130.py:15: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  prediction_list.append(predict_sentiment(prediction_data.iloc[row][0]))


1
1
1
1
1
1
1
0
1
1
1
1
1
1
0
0
1
1
1
1
1
1
1
0
0
1
1
1
1
1
1
1
0
1
1
1
1
1
1
1
0
1
1
1
1
1
0
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
0
1
1
1
1
1
1
1
1
1
0
1
1
1
1
0
1
0
1
1
1
1
0
1
1
1
1
1
1
1
0
0
1
0
0
1
1
1
1
1
0
1
1
1
1
1
1
1
1
1
1
1
1
1
1
0
0
1
1
1
1
1
1
1
1
1
1
1
1
0
1
0
1
1
0
1
1
0
1
1
1
0
1
1
1
1
1
1
1
1
1
0
1
1
1
1
1
1
1
1
1
0
1
1
1
1
1
0
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
0
1
1
1
1
1
1
1
1
0
0
1
1
1
0
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
0
1
1
1
1
1
0
1
1
1
1
1
1
1
1
1
1
1
1
1
0
1
1
1
1
1
1
1
1
1
1
1
1
1
1
0
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
0
0
1
0
1
0
0
1
1
1
1
0
0
1
1
0
1
1
1
1
0
1
1
1
1
0
1
1
1
1
1
1
1
1
0
1
1
0
1
0
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
0
1
1
1
0
1
1
1
1
1
0
1
1
1
1
1
1
1
1
1
1
1
1
1
0
1
0
1
1
0
0
0
1
1
1
1
1
1
1
1
1
1
1
1
0
1
1
1
1
1
1
1
1
1
0
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
0
1
1
1
0
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
0
0
1
1
1
1
1
1
1
1
1
1
1
1
1
0
1
1
1
0
1
1
1
1
1
1
1
1
1
1
1
1
0
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1


In [36]:
# Save the new datafram into a json file
finalized_prediction.to_json('C:/Users/koari/OneDrive/Desktop/IE4483 Proj/output.json',orient='records' ,lines='true')